In [1]:
import numpy as np
import math as m
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.cross_validation import KFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

def calculate_mean_variance(X_train,Y_train):
    ind= range(len(Y_train))
    d={"x1":pd.Series(X_train[:,0], index=ind),"x2":pd.Series(X_train[:,1], index=ind), \
       "x3":pd.Series(X_train[:,2], index=ind),"x4":pd.Series(X_train[:,3], index=ind),"Label":pd.Series(Y_train)}
    df = pd.DataFrame(d)
    sf=pd.DataFrame(columns=('x1', 'x2', 'x3','x4'))
    vf=pd.DataFrame(columns=('x1', 'x2', 'x3','x4'))
    vgf=pd.DataFrame(columns=('x1', 'x2', 'x3','x4'))
    for i in range(len(Y_train)):
        if(df.Label[i] == 1):
            sf.loc[i]=[df.x1[i],df.x2[i],df.x3[i],df.x4[i]]
        elif(df.Label[i]==2):
            vf.loc[i]=[df.x1[i],df.x2[i],df.x3[i],df.x4[i]]
        else:
            vgf.loc[i]=[df.x1[i],df.x2[i],df.x3[i],df.x4[i]]
    prior_setosa= float(len(sf))/float(len(df))
    prior_versi=float(len(vf))/float(len(df))
    prior_vergi=float(len(vgf))/float(len(df))
    cov_setosa=np.cov(sf.transpose())
    cov_versi=np.cov(vf.transpose())
    cov_vergi=np.cov(vgf.transpose())
    mean_setosa=[np.mean(sf.x1),np.mean(sf.x2),np.mean(sf.x3),np.mean(sf.x4)]
    mean_versi=[np.mean(vf.x1),np.mean(vf.x2),np.mean(vf.x3),np.mean(vf.x4)]
    mean_vergi=[np.mean(vgf.x1),np.mean(vgf.x2),np.mean(vgf.x3),np.mean(vgf.x4)]
#     print prior_setosa, prior_versi, prior_vergi, len(sf), len(df)
    return mean_setosa,mean_versi,mean_vergi,cov_setosa,cov_versi,cov_vergi,prior_setosa,prior_versi,prior_vergi

def member(mean_setosa,mean_versi,mean_vergi,cov_setosa,cov_versi,cov_vergi,prior_setosa,prior_versi,prior_vergi,X_train):
    g1=[]
    g2=[]
    g3=[]
    d=pd.DataFrame(columns=('g1', 'g2','g3'))
    for i in range(len(X_train)):
        g1x= -m.log(np.linalg.det(cov_setosa)) - (0.5 *(np.dot((X_train[i]-mean_setosa).transpose(),np.dot(np.linalg.inv(cov_setosa),\
                                                      (X_train[i]-mean_setosa))))) + m.log(prior_setosa)
        g1.append(g1x)
        g2x= -m.log(np.linalg.det(cov_versi)) - (0.5 *(np.dot((X_train[i]-mean_versi).transpose(),np.dot(np.linalg.inv(cov_versi),\
                                                      (X_train[i]-mean_versi))))) + m.log(prior_versi)
        g2.append(g2x)
        g3x= -m.log(np.linalg.det(cov_vergi)) - (0.5 *(np.dot((X_train[i]-mean_vergi).transpose(),np.dot(np.linalg.inv(cov_vergi),\
                                                      (X_train[i]-mean_vergi))))) + m.log(prior_vergi)
        g3.append(g3x)
        
        d.loc[i]=[g1[i],g2[i],g3[i]]
    return d
    
def classify(d,Fun1,Y_train):
    with_max=pd.DataFrame(columns=('g1', 'g2','g3','PLabel','FunLabel','ALabel'))
    for i in range(len(d)):
        maxv=(max(d.loc[i]))
        if(maxv==d.g1[i]):
            cl=1
        elif(maxv==d.g2[i]):
            cl=2
        else:
            cl=3
        with_max.loc[i]=[d.g1[i],d.g2[i],d.g3[i],cl,Fun1[i],Y_train[i]]
    return with_max
   
def confusion_mat(c):
#     print c
#     A=c.ALabel.value_counts()
#     P=c.PLabel.value_counts()
    mse=mean_squared_error(c.ALabel, c.PLabel)
    print "Mean Squared Error: ", mse
    E11=0
    E22=0
    E33=0
    E12=0
    E13=0
    E21=0
    E23=0
    E31=0
    E32=0
    for i in range(len(c)):
        if (c.ALabel[i]== c.PLabel[i] and c.ALabel[i]==1):
            E11+=1
        if (c.ALabel[i]== c.PLabel[i] and c.ALabel[i]==2):
            E22+=1
        if (c.ALabel[i]== c.PLabel[i] and c.ALabel[i]==3):
            E33+=1
        if(c.ALabel[i]==2 and c.PLabel[i]==1):
            E12+=1
        if(c.ALabel[i]==3 and c.PLabel[i]==1):
            E13+=1  
        if(c.ALabel[i]==1 and c.PLabel[i]==2):
            E21+=1
        if(c.ALabel[i]==3 and c.PLabel[i]==2):
            E23+=1 
        if(c.ALabel[i]==1 and c.PLabel[i]==3):
            E31+=1
        if(c.ALabel[i]==2 and c.PLabel[i]==3):
            E32+=1 
#     print count1,count2,count3
    con_mat=np.matrix([[E11,E12,E13],[E21, E22, E23],[E31, E32, E33]])
    p1=float(E11)/float(E11+E12+E13)
    p2=float(E22)/float(E21+E22+E23)
    p3=float(E33)/float(E31+E32+E33)
    r1=float(E11)/float(E11+E21+E31)
    r2=float(E22)/float(E12+E22+E32)
    r3=float(E33)/float(E13+E23+E33)
    precision=[p1,p2,p3]
    recall=[r1,r2,r3]
    Accuracy= float(np.trace(con_mat) / float(E11+E12+E13+E21+E22+E23+E31+E32+E33))
    f1=2*(float((p1*r1))/float((p1+r1)))
    f2=2*(float((p2*r2))/float((p2+r2)))
    f3=2*(float((p3*r3))/float((p3+r3)))
    print "Confusion Matrix: ",con_mat
    print "Precision: ",precision
    print "Recall: " ,recall
    print "F_measure: ",[f1,f2,f3]
    print "Accuracy: ",Accuracy
    
    
def k_fold(X, Y):
    kf=sklearn.cross_validation.KFold(n=len(X), n_folds=10, shuffle=False,random_state=None)
    for train_index, test_index in kf:
        X_train,X_test=X[train_index], X[test_index]
        Y_train,Y_test=Y[train_index], Y[test_index]
        mean_setosa,mean_versi,mean_vergi,cov_setosa,cov_versi,cov_vergi,prior_setosa,prior_versi,prior_vergi=calculate_mean_variance(X_train,Y_train)
        d=member(mean_setosa,mean_versi,mean_vergi,cov_setosa,cov_versi,cov_vergi,prior_setosa,prior_versi,prior_vergi,X_train)
        lda = LinearDiscriminantAnalysis()
        clf = lda.fit(X_train,Y_train)
        Fun1=clf.predict(X_train)
        with_max=classify(d,Fun1,Y_train)
        confusion_mat(with_max)
#     print with_max[0:50]
#     print with_max[50:100]
#     print with_max[100:149]

def main():
    iris = load_iris()
    X = iris.data 
    Y = iris.target
    enc = LabelEncoder()
    label_encoder = enc.fit(Y)
    y = label_encoder.transform(Y) + 1
#  without fold
    print("Performance with 75% training size")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    mean_setosa,mean_versi,mean_vergi,cov_setosa,cov_versi,cov_vergi,prior_setosa,prior_versi,prior_vergi=calculate_mean_variance(X_train,y_train)
    d=member(mean_setosa,mean_versi,mean_vergi,cov_setosa,cov_versi,cov_vergi,prior_setosa,prior_versi,prior_vergi,X_test)
    lda = LinearDiscriminantAnalysis()
    clf = lda.fit(X_train,y_train)
    Fun1=clf.predict(X_train)
    with_max=classify(d,Fun1,y_test)
    confusion_mat(with_max)
# with fold
    print("******")
    print("Performance with 10 fold")
    k_fold(X,y)
    

    
if __name__ == "__main__":main()

Performance with 75% training size
Mean Squared Error:  0.0
Confusion Matrix:  [[15  0  0]
 [ 0 11  0]
 [ 0  0 12]]
Precision:  [1.0, 1.0, 1.0]
Recall:  [1.0, 1.0, 1.0]
F_measure:  [1.0, 1.0, 1.0]
Accuracy:  1.0
******
Performance with 10 fold
Mean Squared Error:  0.0148148148148
Confusion Matrix:  [[35  0  0]
 [ 0 49  1]
 [ 0  1 49]]
Precision:  [1.0, 0.98, 0.98]
Recall:  [1.0, 0.98, 0.98]
F_measure:  [1.0, 0.98, 0.98]
Accuracy:  0.985185185185
Mean Squared Error:  0.0148148148148
Confusion Matrix:  [[35  0  0]
 [ 0 49  1]
 [ 0  1 49]]
Precision:  [1.0, 0.98, 0.98]
Recall:  [1.0, 0.98, 0.98]
F_measure:  [1.0, 0.98, 0.98]
Accuracy:  0.985185185185
Mean Squared Error:  0.0148148148148
Confusion Matrix:  [[35  0  0]
 [ 0 49  1]
 [ 0  1 49]]
Precision:  [1.0, 0.98, 0.98]
Recall:  [1.0, 0.98, 0.98]
F_measure:  [1.0, 0.98, 0.98]
Accuracy:  0.985185185185
Mean Squared Error:  0.0148148148148
Confusion Matrix:  [[45  0  0]
 [ 0 39  1]
 [ 0  1 49]]
Precision:  [1.0, 0.975, 0.98]
Recall:  [1.0,

C:\Users\nivedita.kalele\Anaconda2\lib\site-packages\sklearn\discriminant_analysis.py:453: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
